**Helper Functions**

Below are some helper functions for the simulations. This cell is required for both types of simulations.

In [ ]:
import numpy as np
import progressbar
import matplotlib.pyplot as plt
import progressbar

"""
Generate training/testing data based on parameters.
 
Parameters
--------------------
    n -- int, number of points in data set
    d -- int, data dimension
    W -- numpy array of shape (d,1), true weights of the data distribution
Returns
--------------------
    X -- numpy array of shape (n,d), each row is a data point x_i ~ N(0,I_d)
    Y -- numpy array of shape (n,1), each value is y_i ~ xW + noise
"""
def generateData(n,d,W):
    X = np.random.normal(0,1, size=(n,d))
    noise = np.random.normal(0,0.1,size=(n,1))
    Y = np.dot(X,W) + noise
    return [X,Y]

"""
Computes true value of critical learning rate

Parameters
--------------------
    x      -- numpy array of shape (n,d)
    y      -- numpy array of shape (n,1)
    W      -- numpy array of shape (d,1), true weights of the data distribution
    W_pred -- numpy array of shape (d,1), estimated weights of the data distribution
Returns
--------------------
    alpha  -- float, critical value of the learning rate (computed using true weights)
"""
def calcAlpha1(x,y,W,W_pred):
    xv = np.dot(x,W_pred-W)
    xtxv = np.dot(x.T,xv)
    num = 2 * np.linalg.norm(xv)**2
    den = np.linalg.norm(xtxv)**2
    alpha = num/den
    return alpha


"""
Computes approximate value of critical learning rate

Parameters
--------------------
    x      -- numpy array of shape (n,d)
    y      -- numpy array of shape (n,1)
    W      -- numpy array of shape (d,1), true weights of the data distribution (not used)
    W_pred -- numpy array of shape (d,1), estimated weights of the data distribution
Returns
--------------------
    alpha  -- float, critical value of the learning rate (computed using approximations)
"""
def calcAlpha2(x,y,W,W_pred):
    y_pred = np.dot(x,W_pred)
    num = 2*(np.linalg.norm(y)**2 + np.linalg.norm(y_pred)**2 - 2*np.dot(y.T,y_pred))

    xt_y = np.dot(x.T,y)
    xt_y_pred = np.dot(x.T,y_pred)
    den = np.linalg.norm(xt_y)**2 + np.linalg.norm(xt_y_pred)**2 - 2*np.dot(xt_y.T,xt_y_pred)

    alpha = num/den
    return alpha[0][0]

**Simulation 1**

This simulation implements gradient descent with the approximate learning rate scheduler as well as the analytic solution for increasing values of the training data size.

In [ ]:
n = 1500 # maximum number of training data points
d = 1000 # dimension of input
test_n = 400 # number of data points in test set
numIters = 3000 # number of gradient descent updates
r = 0.9 # parameter in the learning rate scheduler
numSims = 1 # number of times to run the simulation

W = np.random.uniform(0,1, size=(d,1))
X_test,Y_test = generateData(test_n,d,W)

numTrain = [i for i in range(1,n+1)]
analyticRisks = [[0 for i in range(n)] for j in range(numSims)]
gdRisks = [[0 for i in range(n)] for j in range(numSims)]

pb1 = progressbar.ProgressBar()

for sim in pb1(range(numSims)):
    
    X,Y = generateData(n,d,W) # Generate training data for every simulation
    
    pb2 = progressbar.ProgressBar()
    for i in pb2(range(1,n+1)):
        # Extract only i training samples
        x = X[:i,] 
        y = Y[:i,]

        # Analytic Solution
        W_pred = np.dot(np.linalg.pinv(x),y)
        Y_test_pred = np.dot(X_test, W_pred)
        analyticRisks[sim][i-1] = (1/test_n) * np.linalg.norm(Y_test - Y_test_pred)

        # Gradient Descent Solution
        W_pred = np.zeros((d,1))
        for numIter in range(numIters):
            alpha = r * calcAlpha2(x,y,W,W_pred)
            W_pred = W_pred - alpha * np.dot(x.T, np.dot(x,W_pred) - y)
        Y_test_pred = np.dot(X_test,W_pred)
        gdRisks[sim][i-1] = (1/test_n) * np.linalg.norm(Y_test - Y_test_pred)

analyticRisksMean = np.mean(analyticRisks, axis=0)
gdRisksMean = np.mean(gdRisks, axis=0)

In [ ]:
# Plotting test risk against number of training samples

plt.title('Test Risk v/s Number of Training Samples, d = ' + str(d))
plt.plot(analyticRisksMean[:], label='Analytic Test Risk')
plt.plot(gdRisksMean[:], label = 'GD Test Risk')
plt.legend()
plt.show()

**Simulation 2**

This implements the learning rate scheduler (exact or approximate as desired) along with the analytical solution for a selected value of n and d.

In [ ]:
#####################################################################################
# Setting Initial Conditions
# Vary values as desired

# Set to true to simulate exact learning rate scheduler
# Set to false to simulate the approximate learning rate scheduler
isExact = False

d = 1000 # Data Dimension
gamma = 1.15 # Underparametrization Ratio, gamma = 1 implies N = d
N = int(gamma * d) # Number of training
test_n = 500 # Size of test set in the simulations
numSims = 1 # Number of simulations to average results over
numIters = 3000 # Number of iterations in gradient descent
alpha_factors = [0.9,1,1.1] # Corresponds to r in the paper
zero_weights = True # False corresponds to Xavier initialization
doPlots = True # True if you want to display the plots
savePlots = False # True if you want to save the plots in addition to displaying it
#####################################################################################


# Handle whether to use exact or approximate learning rate scheduler
def chooseMethod(isExact):
    x = None
    method = None
    calcAlpha = None

    if isExact:
        method = 'Exact '
        x = '1'
        calcAlpha = calcAlpha1
    else:
        method = 'Approx. '
        x = '2'
        calcAlpha = calcAlpha2
    return [method, x, calcAlpha]

In [ ]:
method, x, calcAlpha = chooseMethod(isExact)

W = np.random.normal(0,1, size=(d,1)) # Initializing true weights for simulation

X_test, Y_test = generateData(test_n,d,W) # Generating test set

numAlphas = len(alpha_factors)

iters = [iter for iter in range(1,numIters+2)]

test_errors_all = np.zeros((numAlphas,numIters+1))
alphas_all = np.zeros((numAlphas,numIters))

pb1 = progressbar.ProgressBar()
for sim in pb1(range(numSims)):
    alphas = [[None for iter in range(numIters)] for i in range(numAlphas)]
    test_errors = [[None for iter in range(numIters+1)] for i in range(numAlphas)]
    X_train,Y_train = generateData(N,d,W) # Generating new training data for every sim
    for i in range(numAlphas):
    # Either zero initialization or Xavier initialization
        W_pred = None
        if zero_weights:
            W_pred = np.zeros((d,1))
        else:
            W_pred = np.random.normal(0,1/(d**0.5), size=(d,1))
        pb2 = progressbar.ProgressBar()

        for iter in range(numIters):
            alpha = alpha_factors[i] * calcAlpha(X_train,Y_train,W,W_pred)
            alphas[i][iter] = alpha
            W_pred = W_pred - alpha * np.dot(X_train.T, np.dot(X_train,W_pred) - Y_train)
            Y_test_pred = np.dot(X_test,W_pred)
            test_error = (1/test_n) * np.linalg.norm(Y_test - Y_test_pred)
            test_errors[i][iter] = test_error
    
        W_pred_2 = np.dot(np.linalg.pinv(X_train),Y_train)
        Y_test_pred = np.dot(X_test,W_pred_2)
        test_error = (1/test_n) * np.linalg.norm(Y_test - Y_test_pred)
        test_errors[i][numIters] = test_error
  
    test_errors = np.array(test_errors)
    test_errors_all = test_errors_all + test_errors
    alphas_all = alphas_all + np.array(alphas)

test_errors_all = (1/numSims) * test_errors_all
alphas_all = (1/numSims) * alphas_all

analytic_risk = np.mean(test_errors_all[:,-1])

down = 0
up = numIters

In [ ]:
# Plotting test risk and optimal learning rate against number of gradient descent updates

down = 0
up = numIters
if doPlots:  
    # Plotting Test Errors
    for i in range(numAlphas-1):
        plt.plot(iters[down:up],test_errors_all[i][down:up],label='r = ' + str(alpha_factors[i]))
    plt.hlines(analytic_risk,down,up,label='Closed Form Solution',color='red')
    plt.title(method + 'learning rate scheduler (n = ' + str(N) + ', d = ' + str(d) + ')')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Test Risk')
    plt.legend()
    if savePlots: plt.savefig('fig' + x + 'a.png')
    plt.show()

    up = 1500
    plt.plot(iters[down:up],test_errors_all[numAlphas-1][down:up],label='r = ' + str(alpha_factors[numAlphas-1]),color='green')
    plt.title(method + 'learning rate scheduler (n = ' + str(N) + ', d = ' + str(d) + ')')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Test Risk')
    plt.legend()
    if savePlots: plt.savefig('fig' + x + 'b.png')
    plt.show()

    # Plotting Learning Rates
    up = numIters
    for i in range(numAlphas-1):
        plt.plot(iters[down:up],alphas_all[i][down:up],label='r = ' + str(alpha_factors[i]))
    plt.title(method + 'learning rate scheduler (n = ' + str(N) + ', d = ' + str(d) + ')')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Learning Rate')
    plt.legend()
    if savePlots: plt.savefig('fig' + x + 'c.png')
    plt.show()

    up = 1500
    plt.plot(iters[down:up],alphas_all[numAlphas-1][down:up],label='r = ' + str(alpha_factors[numAlphas-1]),color='green')
    plt.title(method + 'learning rate scheduler (n = ' + str(N) + ', d = ' + str(d) + ')')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Learning Rate')
    plt.legend()
    if savePlots: plt.savefig('fig' + x + 'd.png')
    plt.show()